# Convolutional Nerual Network 🧠
### Transfer Learning
- 작은 이미지 데이터셋에 딥러닝을 적용하는 일반적이고 효과적인 방법은 **사전 훈련된 네트워크** 사용 
- **사전 훈련된 네트워크(pretrained network)** : 일반적으로 대규모 이미지 분류 문제를 위해 대량의 데이터 셋에서 미리 훈련되어 저장된 네트워크 
  - ImageNet : 1400만개의 레이블된 이미지와 1000개의 클래스로 이루어 진 데이터셋 📷

#### 사전 훈련된 네트워크 사용하는 방법❓
  - **Fine Tuning(미세조정)**
    - 특성추출에서 사용했던 동결모델의 상위 층 몇개를 동결에서 해제하고 모엘데 새로 추가한 층(완전 연결분류기)와 함께 연결

In [ ]:
import warnings 
warnings.filterwarnings(action='ignore') 

In [ ]:
# 파일 압출 해제 
!unzip cats_and_dogs_small.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: cats_and_dogs_small/train/cats/cat.333.jpg  
  inflating: __MACOSX/cats_and_dogs_small/train/cats/._cat.333.jpg  
  inflating: cats_and_dogs_small/train/cats/cat.864.jpg  
  inflating: __MACOSX/cats_and_dogs_small/train/cats/._cat.864.jpg  
  inflating: cats_and_dogs_small/train/cats/cat.870.jpg  
  inflating: __MACOSX/cats_and_dogs_small/train/cats/._cat.870.jpg  
  inflating: cats_and_dogs_small/train/cats/cat.680.jpg  
  inflating: __MACOSX/cats_and_dogs_small/train/cats/._cat.680.jpg  
  inflating: cats_and_dogs_small/train/cats/cat.858.jpg  
  inflating: __MACOSX/cats_and_dogs_small/train/cats/._cat.858.jpg  
  inflating: cats_and_dogs_small/train/cats/cat.99.jpg  
  inflating: __MACOSX/cats_and_dogs_small/train/cats/._cat.99.jpg  
  inflating: cats_and_dogs_small/train/cats/cat.694.jpg  
  inflating: __MACOSX/cats_and_dogs_small/train/cats/._cat.694.jpg  
  inflating: cats_and_dogs_small/train/cats/cat.72.jpg  
  inflating: __MACOS

In [ ]:
import os

original_db_dir = './train'
base_dir ='./cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
from keras.applications.vgg16 import VGG16

conv_base= VGG16(
    weights= 'imagenet',        # 가중치 사용 여부 
    include_top = False,        # 맨 마지막인 Classifier는 사용하지 않는다
    input_shape=(150,150,3)
)

conv_base.summary()

58900480/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
______________________________________________________________

In [ ]:
from keras.layers import Dense, Dropout
from keras import models 

model = models.Sequential()
model.add(Dense(256, activation='relu', input_dim=4*4*512))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
conv_base.trainable = True 

set_trainable = False
for layer in conv_base.layers:
  if layer.name =='block5_conv1':
    set_trainable = True 
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20
train_datagen = ImageDataGenerator(
    rescale = 1. /255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode ='binary'
)

Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
# model compile 
from tensorflow.keras import optimizers

model.compile(optimizer= optimizers.RMSprop(lr=1e-5),
              loss = 'binary_crossentropy',
              metrics=['acc'])

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 100,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=50
)

Epoch 1/100


ValueError: ignored